In [1]:
using Flight.FlightCore.Systems
using Flight.FlightCore.Plotting

using Flight.FlightPhysics.Geodesy
using Flight.FlightPhysics.Kinematics

using Flight.FlightComponents.Aircraft

using Flight.FlightAircraft.C172R

using ControlSystems

ERROR: LoadError: 

syntax: "c::PIDDiscrete{1} = PIDDiscrete{1}(k_p = 10, k_i = 25, k_d = 5, τ_d = 0.05)" inside type definition is reserved around c:\Users\Miguel\.julia\dev\Flight\src\aircraft\c172r\variants\c172rv2.jl:144
Stacktrace:
  [1] top-level scope
    @ c:\Users\Miguel\.julia\dev\Flight\src\aircraft\c172r\variants\c172rv2.jl:144
  [2] include(mod::Module, 

_path::String)
    @ Base .\Base.jl:457
  [3] include(x::String)
    @ Flight.FlightAircraft.C172R c:\Users\Miguel\.julia\dev\Flight\src\aircraft\c172r\c172r.jl:1
  [4] top-level scope
    @ c:\Users\Miguel\.julia\dev\Flight\src\aircraft\c172r\c172r.jl:8
  [5] include(mod::Module, _path::String)
    @ Base .\Base.jl:457
  [6] include(x::String)
    @ Flight.FlightAircraft c:\Users\Miguel\.julia\dev\Flight\src\aircraft\FlightAircraft.jl:1
  [7] top-level scope
    @ c:\Users\Miguel\.julia\dev\Flight\src\aircraft\FlightAircraft.jl:5
  [8] include(mod::Module, _path::String)
    @ Base .\Base.jl:457
  [9] include(x::String)
    @ Flight c:\Users\Miguel\.julia\dev\Flight\src\Flight.jl:1
 [10] top-level scope
    @ c:\Users\Miguel\.julia\dev\Flight\src\Flight.jl:8
 [11] include
    @ .\Base.jl:457 [inlined]
 [12] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgI

ErrorException: Failed to precompile Flight [e329eb8f-69f3-4234-a468-5e3889a58643] to "C:\\Users\\Miguel\\.julia\\compiled\\v1.9\\Flight\\jl_426.tmp".

In [2]:
ac = Cessna172Rv0(NED()) |> System #linearization requires NED kinematics
ac.u.airframe.pld.m_pilot = 75
ac.u.airframe.pld.m_copilot = 75
ac.u.airframe.pld.m_lpass = 0
ac.u.airframe.pld.m_rpass = 0
ac.u.airframe.pld.m_baggage = 0

trim_params = C172Rv0.TrimParameters(
    loc = LatLon(),
    h = HOrth(1000),
    ψ_nb = 0.0,
    TAS = 40.0,
    γ_wOb_n = 0.0,
    ψ_lb_dot = 0.00,
    θ_lb_dot = 0.0,
    β_a = 0.0,
    fuel = 1,
    mixture = 0.5,
    flaps = 0.0)

lm = Aircraft.linearize!(ac; trim_params)

UndefVarError: UndefVarError: `NED` not defined

## Lateral Dynamics

In [3]:
lat_dyn = filter(lm;
    u = (:aileron, :rudder),
    x = (:v_y, :φ, :p, :r, :β_filt),
    y = (:p, :r, :φ, :ψ, :β, :f_y))

lat_dyn_ss = lat_dyn |> ss
lat_dyn_tf = lat_dyn_ss |> tf

UndefVarError: UndefVarError: `lm` not defined

### Roll Dynamics

In [4]:
roll_dyn = filter(lat_dyn;
    u = (:aileron,),
    y = (:p,))

roll_dyn_ss = roll_dyn |> ss
roll_dyn_tf = roll_dyn_ss |> tf
@show zpk(roll_dyn_tf)

UndefVarError: UndefVarError: `lat_dyn` not defined

In [5]:
#there is a natural pole-zero near-cancellations. it is in the right half plane,
#but its time constant is extremely slow. since they are natural (not achieved
#through compensation), we can probably simplify them. they likely represent
#nearly uncontrollable states, because the pole is insensitive to feedforward
#compensation. this pole-zero pair disappears when v_y is omitted from the
#lateral dynamics, so it may represent the intuitive notion that v_y is not
#really controllable in a conventional aircraft! it is not mathematical, exact
#uncontrollability (which would result in exact pole-zero cancellation), but
#practical uncontrollability
a2p = minreal(roll_dyn_tf, 1e-2) 
@show zpk(a2p)

#output error transfer function for a unit feedback closed loop
err_a2p = 1/(1+a2p)
@show zpk(err_a2p)
#steady state error of the closed loop system for a unit step input
@show err_a2p(0)
step(err_a2p, 10) |> plot


UndefVarError: UndefVarError: `minreal` not defined

In [6]:
#define a PID compensator
k_p = 0.5
k_i = 10
k_d = 0.05
τ_d = 0.05
p_cmp = k_p + k_i * tf(1, [1,0]) + k_d * tf([1, 0], [τ_d, 1])
    
c_a2p = series(p_cmp, a2p)
err_c_a2p = 1/(1+c_a2p)
@show zpk(err_c_a2p);
marginplot(c_a2p)
# nyquistplot(c_a2p)

UndefVarError: UndefVarError: `tf` not defined

In [7]:
step(err_c_a2p, 10) |> plot

UndefVarError: UndefVarError: `err_c_a2p` not defined

Let's check the response we get from the full plant, without the zero-pole simplification

In [8]:
c_roll_dyn = series(p_cmp, roll_dyn_tf)

err_c_roll_dyn = 1/(1+c_roll_dyn)
@show zpk(err_c_roll_dyn)
# @show err_c_yaw_dyn(0)

# marginplot(c_yaw_dyn)

UndefVarError: UndefVarError: `series` not defined

In [9]:
step(err_c_roll_dyn, 10) |> plot

UndefVarError: UndefVarError: `err_c_roll_dyn` not defined

Now we have a SISO compensator design for p, let's use it to close the loop in the MIMO plant and make sure we get the expected response. The lat_dyn MIMO system has two inputs, aileron input u_a and rudder input u_r. Our SISO compensator receives p_err as an input and outputs u_a. So the MIMO version of our SISO compensator, which will be placed upstream of the MIMO plant, must receive p_err and u_r as inputs and output u_a and u_r. Therefore, it must apply the SISO compensator to p_err to obtain u_a and let u_r pass through unchanged to the plant by applying an unit gain to it. This is achieved as follows:

In [10]:
#create a diagonal MIMO system from the roll rate compensator and a unit
#transfer function
p_cmp_MIMO = append(p_cmp, tf(1)) |> ss #and convert to ss before concatenation

UndefVarError: UndefVarError: `p_cmp` not defined

In [11]:
p_cmp_lat_dyn = series(p_cmp_MIMO, lat_dyn_ss) #equivalent to long_dyn_ss * q_comp_MIMO (in the product notation, the second system goes first)

UndefVarError: UndefVarError: `series` not defined

The setpoint vector to be applied to the complete closed-loop MIMO system is r = [p_cmd, u_r]. The input to the MIMO compensator must be e = [p_err, u_r] = r - f = [p_cmd, u_r] - [p, 0]. So the feedback vector we need is f = [p, 0]. To construct this from the 6-output vector of lat_dyn, we need a matrix gain that premultiplies this output vector and produces a 2-component vector [p, 0]. This will be a 2x6 matrix with all entries set to zero except [1, 1] (the nonzero output p goes in the first component, and p is the first element in lat_dyn's output vector)

In [12]:
K_p = zeros(2, 6)
K_p[1,1] = 1
K_p_ss = ss(K_p) #creates a ss with no dynamics and K_select as feedthrough

#now create a feedback loop with K in the feedback path
p_cmp_lat_dyn_cl = feedback(p_cmp_lat_dyn, K_p_ss)

#now extract the resulting closed-loop roll demand to roll rate transfer function 
p_cl_MIMO = (p_cmp_lat_dyn_cl |> tf)[1,1]

#make sure the zpk data matches that of the SISO closed loop TF
@show zpk(p_cl_MIMO)

#check step response
step(p_cl_MIMO, 10) |> plot


UndefVarError: UndefVarError: `ss` not defined

### Turn Coordination

First we focus on turn coordination. the plan is to design a SISO turn coordinator, ignoring the roll - yaw coupling for the moment. the result will be a SISO compensator with β_err as input and u_r as output. we embed it in a MIMO compensator that applies the SISO compensator to β_err and lets u_a pass through unchanged, and connect it upstream of the lateral dynamics plant. the result is then a plant with β_dmd and u_a as inputs.

From this updated plant, in a second step, we design a SISO roll rate controller with p_err as input and u_a as output. we embed it in a MIMO compensator that has p_err and β_dmd as inputs and u_a and β_dmd as outputs. of course, since roll and yaw are coupled, adding this second controller will alter the β to u_r response that resulted from the previous design, so we will have to check whether it is still satisfactory. if not, further iterations might be required.

Once the sequential design is done, we can encapsulate both SISO controllers in a single MIMO controller with a decoupled internal structure.

Maybe start with roll control. Because it looks like we can't deal with the unstable pole with a rudder input

In [13]:
yaw_dyn = filter(lat_dyn;
    u = (:rudder,),
    y = (:β,))

yaw_dyn_ss = yaw_dyn |> ss
yaw_dyn_tf = yaw_dyn_ss |> tf
@show zpk(yaw_dyn_tf)

UndefVarError: UndefVarError: `lat_dyn` not defined

In [14]:
#get rid of natural zero-pole near-cancellation
r2β = minreal(yaw_dyn_tf, 1e-1)
@show zpk(r2β)

#the first thing we notice is that the DC gain is negative. this makes sense,
#given that we have: act.rudder↑ -> aero.r↓ -> yaw↑ -> β↓. therefore, the first
#thing we need in the forward path is a sign change.
c1 = tf(-1)
c1_r2β = series(c1, r2β)
@show zpk(c1_r2β)

#output error transfer function for a unit feedback closed loop
err_c1_r2β = 1/(1+c1_r2β)
# @show zpk(err_c1_r2β)
@show err_c1_r2β(0)
step(err_c1_r2β, 10) |> plot
#this is type 0 system, so we will need an integrator in the feedforward path


UndefVarError: UndefVarError: `minreal` not defined

In [15]:
#define a PID
k_p = 10
k_i = 25
k_d = 5
τ_d = 0.05
c2 = k_p + k_i * tf(1, [1,0]) + k_d * tf([1, 0], [τ_d, 1])

#the compensator is the PID chained with the sign inversion
β_cmp = series(c1, c2)
    
c_r2β = series(β_cmp, r2β)

err_c_r2β = 1/(1+c_r2β)
# @show zpk(err_c2_c1_r2β)
@show err_c_r2β(0)

marginplot(c_r2β)

UndefVarError: UndefVarError: `tf` not defined

In [16]:
step(err_c_r2β, 10) |> plot

UndefVarError: UndefVarError: `err_c_r2β` not defined

Let's check the response we get from the full plant, without the zero-pole simplification

In [17]:
c_yaw_dyn = series(β_cmp, yaw_dyn_tf)

err_c_yaw_dyn = 1/(1+c_yaw_dyn)
@show zpk(err_c_yaw_dyn)
# @show err_c_yaw_dyn(0)

# marginplot(c_yaw_dyn)

UndefVarError: UndefVarError: `series` not defined

In [18]:
step(err_c_yaw_dyn, 10) |> plot

UndefVarError: UndefVarError: `err_c_yaw_dyn` not defined

As expected, it is quite similar, except for a slowly creeping divergence due to the slow unstable pole, which we haven't dealt with and is less exactly cancelled than in the roll dynamics.

Now we have a SISO compensator design for $\beta$, let's use it to close the loop in the MIMO plant and make sure we get the expected response. The lat_dyn MIMO system has two inputs, aileron input u_a and rudder input u_r. Our SISO compensator receives $e_{\beta}$ as an input and outputs u_r. So the MIMO version of our SISO compensator, which will be placed upstream of the MIMO plant, must receive u_a and beta_err as inputs and output u_a and u_r. Therefore, it must let u_a pass through unchanged to the plant by applying an unit gain to it, and apply the SISO compensator to beta_err to obtain u_r. This is achieved as follows:

In [19]:
#create a diagonal MIMO system from a unit TF and the beta compensator
β_cmp_MIMO = append(tf(1), β_cmp) |> ss #and convert to ss before concatenation

UndefVarError: UndefVarError: `tf` not defined

In [20]:
β_cmp_lat_dyn = series(β_cmp_MIMO, lat_dyn_ss) #equivalent to long_dyn_ss * q_comp_MIMO (in the product notation, the second system goes first)

UndefVarError: UndefVarError: `series` not defined

The setpoint vector to be applied to the complete closed-loop MIMO system is r = [u_a, beta_cmd]. The input to the MIMO compensator must be e = [u_a, beta_err] = r - f = [u_a, beta_cmd] - [0, beta]. So the feedback vector we need is f = [0, beta]. To construct this from the 6-output vector of lat_dyn, we need a matrix gain that premultiplies this output vector and produces a 2-component vector [0, beta]. This will be a 2x6 matrix with all entries set to zero except [2, 5] (the nonzero output beta goes in the second output component, and beta is the 5th element in lat_dyn's output vector)

In [21]:
K_β = zeros(2, 6)
K_β[2,5] = 1
K_β_ss = ss(K_β) #creates a ss with no dynamics and K_select as feedthrough

#now create a feedback loop with K in the feedback path
β_cmp_lat_dyn_cl = feedback(β_cmp_lat_dyn, K_β_ss)

#now extract the resulting closed-loop beta demand to beta transfer function 
β_cl_MIMO = (β_cmp_lat_dyn_cl |> tf)[5,2]

#make sure the zpk data matches that of the SISO closed loop TF
@show zpk(β_cl_MIMO)

#check step response
step(β_cl_MIMO, 10) |> plot


UndefVarError: UndefVarError: `ss` not defined

### Roll control and turn coordination

In [22]:
#create a diagonal MIMO system from both compensators
pβ_cmp_MIMO = append(p_cmp, β_cmp) |> ss #and convert to ss before concatenation

UndefVarError: UndefVarError: `append` not defined

In [23]:
pβ_cmp_lat_dyn = series(pβ_cmp_MIMO, lat_dyn_ss) #equivalent to long_dyn_ss * q_comp_MIMO (in the product notation, the second system goes first)

UndefVarError: UndefVarError: `series` not defined

In [24]:
K_pβ = zeros(2, 6)
K_pβ[1,1] = 1
K_pβ[2,5] = 1
K_pβ_ss = ss(K_pβ) #creates a ss with no dynamics and K_select as feedthrough

#now create a feedback loop with K in the feedback path
pβ_cmp_lat_dyn_cl = feedback(pβ_cmp_lat_dyn, K_pβ_ss)

#now extract the resulting closed-loop SISO transfer functions
p_cl_MIMO = (pβ_cmp_lat_dyn_cl |> tf)[1,1] #input 1 to output 1
β_cl_MIMO = (pβ_cmp_lat_dyn_cl |> tf)[5,2] #input 2 to output 5

#make sure the zpk data matches that of the SISO closed loop TF
@show zpk(p_cl_MIMO)
@show zpk(β_cl_MIMO)

UndefVarError: UndefVarError: `ss` not defined

From the closed loop transfer functions, we see that with both controllers are implemented and operating in parallel as a MIMO compensator, the slow unstable pole is nearly perfectly cancelled in the full lateral dynamics!

In [25]:
#check roll rate demand step response
step(p_cl_MIMO, 10) |> plot

UndefVarError: UndefVarError: `p_cl_MIMO` not defined

In [26]:
#check β demand step response
step(β_cl_MIMO, 10) |> plot

UndefVarError: UndefVarError: `β_cl_MIMO` not defined